In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import html5lib
import re
from datetime import datetime

## Get unincorporated area boundaries (Do only once)

from lxml import html 
import time

#Get page of links to each unincorporated area
unincorpurl = 'http://maps.latimes.com/neighborhoods/unincorporated/list/page/1/'
unincorpresponse = requests.get(unincorpurl)
unincorpsoup = BeautifulSoup(unincorpresponse.text, "html.parser")

#Get hyperlinks
alist = unincorpsoup.findAll('a', href=True)

#Loop over each unincorporated area
count = 0
outfile = "data/unincorporated_latimes.txt" #Change to .geojson.  Also fix formatting at a later date
for link in alist:
    if ('Unincorporated' in str(link)):
        print('Working on ', link)
        download_url = 'http://maps.latimes.com/'+ link['href']
        
        pageContent=requests.get(download_url) #Use lxml to be able to scrape javascript.         
        tree = html.fromstring(pageContent.content)
        js=tree.xpath('//*[@id="content"]/div[1]/script/text()') #XPath for the script
        result = re.search('features": \[(.*)]    };    ', str(js[0].replace('\n', ''))) #Extract geoJSON feature
        
        with open(outfile, "a+") as text_file:
            if count < 1 :
                text_file.write('{\n "type": "FeatureCollection", \n "features": [\n') #Untested
            text_file.write(result.group(1)+', \n') #Be sure to go remove the last comma manually and fix the beginning
        time.sleep(1) #pause the code for a sec
        count = count + 1
with open(outfile, "a+") as text_file:
    text_file.write('REMOVE PRECEEDING COMMA \n ] }')
        

#Check file output        
import geopandas as gpd
demog = gpd.read_file('data/unincorporated_latimes.geojson')
demog.plot()

## Daily scrape of LA County Public Health table

In [ ]:
#Chose what to scrape
archive = False
titlestring='0625' 

if archive:     
    url = 'data/'+titlestring+'.html' #Uses locally downloaded file from wayback machine to lookup archived site
    soup = BeautifulSoup(open(url),"html.parser")
else:
    url ='http://publichealth.lacounty.gov/media/Coronavirus/locations.htm'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
#    datetime.today().strftime('%Y-%m-%d')
#    titlestring=datetime.today().strftime('%m%d') #Use today's date

In [ ]:
tr_elements = soup.find_all('tr', {'class' : 'blue text-white'})[-1].find_next_siblings()

colnames = ['Locations','Total Cases', 'Rate', 'Deaths', 'Death Rate']
#for col in soup.find_all('tr')[0].find_all('td'):
#    colnames.append(col.text.replace('*', ''))
    
df = pd.DataFrame(columns=colnames, index = range(0,len(tr_elements)+2)) #To add in total and Pasadena

row_marker = 0

for row in tr_elements:
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:        
        df.iat[row_marker,column_marker] = column.get_text().replace('--', 'suppressed').replace('City of ', '').replace('Unincorporated - ', '').replace('Los Angeles - ','').replace('*','')
        column_marker += 1
    row_marker+=1

#Add total - currently hardcoded
column_marker =0 
for column in soup.find_all('tr')[4].find_all('td'): #Changed value to 5. Previously 1?

    if (column_marker==0):
        print('Renaming ', column.get_text(), 'to Total')
        df.iat[-1, column_marker] = 'Total'
    else:
        df.iat[-1, column_marker] = column.get_text()
        print(column.get_text())
    column_marker += 1
    

In [ ]:
df

In [ ]:
#Add Pasadena back in 
column_marker =0
pasadenapop = 141371.
for column in soup.find_all('tr')[11].find_all('td'):
    print(column.get_text())

In [ ]:
#Add Pasadena back in 
#column_marker =0
pasadenapop = 141371.
#for column in soup.find_all('tr')[8].find_all('td'): #Previously 4, change as of May ~15
#    if(column_marker <2):
#        print(column.get_text())
#        df.iat[-3, column_marker] = column.get_text().replace('-','').replace(' ','')
#        if(column_marker ==1):
#            df.iat[-3, column_marker+1] = int( column.get_text().replace('-','')) /pasadenapop *100000
#    else:
#        print('Skipping:', column.get_text())
#    column_marker += 1
df.iat[-3, 0] = soup.find_all('tr')[7].find_all('td')[0].get_text().replace('-','').replace(' ','')
df.iat[-3, 1] = int(soup.find_all('tr')[7].find_all('td')[1].get_text())
df.iat[-3, 2] = int(soup.find_all('tr')[7].find_all('td')[1].get_text())/pasadenapop *100000
df.iat[-3, 3] = int(soup.find_all('tr')[11].find_all('td')[1].get_text())
df.iat[-3, 4] = int(soup.find_all('tr')[11].find_all('td')[1].get_text())/pasadenapop *100000    

In [ ]:
#Add Long Beach back in (No longer in table list like it was before)
#df.at['Long Beach',colnames[1]] = soup.find_all('tr')[7].find_all('td')[1].text.replace('*','') #Previously 3, not 7
#df.at['Long Beach',colnames[2]] = int(soup.find_all('tr')[7].find_all('td')[1].text)/longbeachpop*100000

#column_marker =0
longbeachpop = 467354.
#for column in soup.find_all('tr')[7].find_all('td'): #Previously 4, change as of May ~15
#    if(column_marker <2):
#        print(column.get_text())
#        df.iat[-2, column_marker] = column.get_text().replace('- ','')
#        if(column_marker ==1):
#            df.iat[-2, column_marker+1] = int( column.get_text().replace('- ','')) /longbeachpop *100000
#    else:
#        print('Skipping:', column.get_text())
#    column_marker += 1
df.iat[-2, 0] = 'Long Beach'
df.iat[-2, 1] = int(soup.find_all('tr')[6].find_all('td')[1].get_text())
df.iat[-2, 2] = int(soup.find_all('tr')[6].find_all('td')[1].get_text())/longbeachpop *100000
df.iat[-2, 3] = int(soup.find_all('tr')[10].find_all('td')[1].get_text())
df.iat[-2, 4] = int(soup.find_all('tr')[10].find_all('td')[1].get_text())/longbeachpop *100000

In [ ]:
soup.find_all('tr')[7].find_all('td')[1].get_text()

In [ ]:
df

In [ ]:
#Show which locations have duplicates - due to Unincorporated or City of LA
grouped = df[df['Locations'].duplicated(keep=False)].groupby('Locations')
df[df['Locations'].duplicated(keep=False)]

In [ ]:
#Failed alt methods:
#[['Total Cases', 'Rate']].sum().head(10)
#.agg({'Total Cases' : 'sum'}).head(10)#, 'Rate' : lambda x: x.iloc[n]})

for name, group in grouped:
#    print('Before', group)
    localsum = 0
    localpop = 0
    firstindex = 0
    for row_index, row in group.iterrows():
        if firstindex==0:
            firstindex = row_index
        if (('suppressed' not in row['Total Cases']) and (int(row['Total Cases']) >0)):
            localsum = localsum + int(row['Total Cases'])
            mypop = int(row['Total Cases'])*100000/float(row['Rate'])
            localpop = localpop+mypop
            
#        print(row_index, 'I have ', row['Total Cases'], ' but my city has ', localsum)
#        print(row_index, row['Rate']) 
    if localsum >0:
        localrate = localsum*100000/localpop #This overestimates the rates in communities whose data are suppressed or have 0 confirmed cases
#        print('ROW INDEX', row_index)
        df.iat[row_index,2] = localrate
        df.iat[firstindex,2] = localrate
    else:
        df.iat[row_index,2] = 0
        df.iat[firstindex,2] = 0
    df.iat[row_index,1] = localsum
    df.iat[firstindex,1] = localsum #Note that different arguments for drop_duplicates could avoid needing to update the firstindex versions

In [ ]:
#Check that it updated successfully
df[df['Locations'].duplicated(keep=False)]

In [ ]:
df.drop_duplicates(keep='first', inplace=True)

In [ ]:
df.count() 

In [ ]:
df#.loc[df['Locations'].str.contains('Under Investigation')]

In [ ]:
#Other cleanup - this is different once the index has been changed:
df.loc[df['Locations'].str.contains('Under Investigation'), 'Locations'] = 'Under Investigation'
df.loc[df['Locations'].str.match('Los Angeles'), 'Locations'] = 'Los Angeles - AGGREGATE'
df.loc[df['Locations'].str.match('Laboratory Confirmed Cases (LCC)'), 'Locations'] = 'Total'

df[df['Locations'].str.contains('AGGREGATE')]

In [ ]:
df.set_index('Locations', inplace=True)
df

#Fix Long Beach and Pasadena
column_marker = 0
for column in soup.find_all('tr')[3].find_all('td'):
    print(df.loc['Long Beach'])
    df.at['Long Beach', colnames[column_marker]] = column.get_text()
    #    df.iat[df.loc['Long Beach'] , column_marker] = column.get_text()
    #    print('Before', df[df['Locations'].str.contains('Long Beach')])
    #    df.iat[df.index(df['Locations'].str.contains('Long Beach')).tolist(), column_marker] = column.get_text()
    column_marker += 1
    #    print('After', df[df['Locations'].str.contains('Long Beach')])
    

In [ ]:
df

In [ ]:
df.loc['Long Beach']

In [ ]:
df.loc['Woodland Hills']

In [ ]:
df

In [ ]:
#Write out to CSV. Make the format match what covid_la.ipynb expects
df.index.names=['city']
df.rename(columns={'Locations': 'city', 'Total Cases': 'count', 'Rate': 'rate'}).to_csv(
    "./data/covid_"+titlestring+".csv",
    index=True,
    encoding="utf-8"
)
#THERE IS A KNOWN BUG that Under Investigation is dropped in archived scrapes! Must be readded